In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from pathlib import Path
import pandas as pd
import numpy as np

In [3]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # Inputs to hidden layer
        self.hidden1 = nn.Linear(18, 12)
        # Output layer
        self.output = nn.Linear(12, 1)
        self.sigmoid = nn.Sigmoid()
        self.elu = nn.ELU()
    
    def forward(self, x):
        # Pass the input tensor through the following
        x = self.hidden1(x)
        x = self.elu(x)
        x = self.output(x)
        x = self.sigmoid(x)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
    
net = Net()
print(net)

Net(
  (hidden1): Linear(in_features=18, out_features=12, bias=True)
  (output): Linear(in_features=12, out_features=1, bias=True)
  (sigmoid): Sigmoid()
  (elu): ELU(alpha=1.0)
)


In [36]:
# Define inputs and target
current_dir = Path.cwd() / "college-predictor"
input_df = pd.read_csv(current_dir / "data/working" / "2000-2017-pace.csv", comment = "#")
x = input_df.iloc[:,4:22]
y = input_df.iloc[:,3]
input = torch.tensor(x.values.astype(np.float32))
target = torch.tensor(y.values.astype(np.float32))

In [37]:
# Define loss function
output = net(input)
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7976, grad_fn=<MseLossBackward>)


In [42]:
# Define optimizer
learning_rate = 0.01
optimizer = optim.SGD(net.parameters(), lr = learning_rate)
optimizer.zero_grad()

In [44]:
# Update the weights
epochs = 10
for e in range(epochs):
    running_loss = 0
    output = net(input)  # foward pass
    loss = criterion(output, target)  # calculate loss
    loss.backward() # calculate gradients
    optimizer.step() # update weights with optimizer
    running_loss += loss.item()
print(f"training loss: {running_loss}")

training loss: 0.7976190447807312
